In [ ]:
from matplotlib import pyplot as plt
import os
import datetime

from summer.utils import ref_times_to_dti

from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.core.runs import ManagedRun
from autumn.core.project import get_project, load_timeseries

In [ ]:
region = "northern_territory"

In [ ]:
run_id = "sm_sir/northern_territory/1661084841/49c4217"

In [ ]:
mr = ManagedRun(run_id)

In [ ]:
pbi = mr.powerbi.get_db()
targets = pbi.get_targets()
results = pbi.get_uncertainty()
mcmc_params = mr.calibration.get_mcmc_params()

## Calibration parameters are:

In [ ]:
list(mcmc_params.columns)

In [ ]:
plt.style.use("ggplot")

param_lookup = {
    "contact_rate": "infection risk per contact",
    "voc_emergence.ba_1.cross_protection.ba_2.early_reinfection": "BA.1 protection against BA.2",
}

unit_lookup = {
    "contact_rate": "probability",
    "voc_emergence.ba_1.cross_protection.ba_2.early_reinfection": "multiplier",
}

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
for i_ax, axis in enumerate(axes.reshape(-1)):
    if i_ax < 5:
        param = mcmc_params.columns[i_ax]
        axis.hist(mcmc_params[param])
        axis.set_title(param_lookup[param])
        axis.set_xlabel(unit_lookup[param])
    if i_ax == 5:
        axis.set_axis_off()

fig.suptitle("parameter posterior histograms", fontsize=15, y=1)
fig.tight_layout()

## Available model outputs are:

In [ ]:
project = get_project("sm_sir", region, reload=True)

### Get all targets, including those not used in calibration, to use as a validation

In [ ]:
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
model, country, run, commit = run_id.split("/")
project_file_path = os.path.join(project_root, "autumn", "projects", model, "australia", country, "timeseries.secret.json")
all_targets = load_timeseries(project_file_path)
for target in all_targets:
    all_targets[target].index = ref_times_to_dti(COVID_BASE_DATETIME, all_targets[target].index)

In [ ]:
colours = (
    (0.2, 0.2, 0.8), 
    (0.2, 0.8, 0.2), 
    (0.8, 0.2, 0.2), 
    (0.8, 0.8, 0.2), 
    (0.8, 0.2, 0.8), 
    (0.2, 0.8, 0.8), 
    (0.8, 0.8, 0.8),
)
title_lookup = {
    "notifications": "daily notifications",
    "infection_deaths": "COVID-19-specific deaths",
    "hospital_admissions": "new daily hospital admissions",
    "icu_admissions": "new daily admissions to ICU",
    "proportion_seropositive": "proportion recovered from COVID-19",
    "incidence": "daily new infections",
    "prop_incidence_strain_delta": "proportion of cases due to Delta",
    "hospital_admissions": "daily hospital admissions",
    "hospital_occupancy": "total hospital beds",
    "icu_admissions": "daily ICU admissions",
    "icu_occupancy": "total ICU beds",
    "prop_ever_infected": "ever infected with Delta or Omicron",
}

In [ ]:
def plot_outputs(outputs, left_date, right_date):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharex="all")
    for i_ax, axis in enumerate(axes.reshape(-1)):
        output = outputs[i_ax]
        for scenario in range(3):
            colour = colours[scenario]
            results_df = results[(output, scenario)]
            indices = results_df.index
            interval_label = "baseline" if scenario == 0 else project.param_set.scenarios[scenario - 1]["description"]
            scenario_zorder = 10 if scenario == 0 else scenario
            axis.fill_between(
                indices, 
                results_df[0.025], results_df[0.975], 
                color=colour, 
                alpha=0.5,
                label="_nolegend_",
                zorder=scenario_zorder,
            )
            axis.fill_between(
                indices, 
                results_df[0.25], results_df[0.75], 
                color=colour, alpha=0.7, 
                label=interval_label,
                zorder=scenario_zorder
            )
            axis.plot(indices, results_df[0.500], color=colour)
            if output in all_targets and len(all_targets[output]) > 0:
                all_targets[output].plot.line(
                    ax=axis, 
                    linewidth=0., 
                    markersize=8.,
                    marker="o",
                    markerfacecolor="w",
                    markeredgecolor="w",
                    alpha=0.2,
                    label="_nolegend_",
                    zorder=11,
                )
            if output in targets:
                targets[output].plot.line(
                    ax=axis, 
                    linewidth=0., 
                    markersize=5., 
                    marker="o", 
                    markerfacecolor="k",
                    markeredgecolor="k",
                    label="_nolegend_",
                    zorder=12,
                )
            axis.tick_params(axis="x", labelrotation=45)
            axis.set_xlim(
                left=left_date, 
                right=right_date
            )
            axis.set_title(title_lookup[output])
        if i_ax == 0:
            axis.legend()
    fig.tight_layout()

In [ ]:
outputs_to_plot = (
    "notifications", 
    "infection_deaths",
    "hospital_admissions",
    "icu_occupancy",
)
plot_outputs(
    outputs_to_plot, 
    datetime.date(2021, 12, 1),
    datetime.date(2022, 7, 1),
)